# Background

Perhaps you've played [Twenty Questions](https://en.wikipedia.org/wiki/Twenty_Questions) before:  it's a game where one player (the *answerer*) thinks of a person, place, or thing, and other players ask yes-or-no questions to guess the object of the answerer's thoughts.  Since the answerer probably knows about a lot of different people and objects, a good strategy for the other players involves devising questions that reduce the space of possible answers as much as possible no matter how they are answered.  

Given a labeled collection of examples, you might imagine a technique to [learn a *decision tree*](https://en.wikipedia.org/wiki/Decision_tree_learning) of questions to classify these examples by asking as few questions as possible.  However, you might imagine that such a technique would necessarily be quite dependent on the exact examples on offer.  (In other words,  these techniques are prone to *overfitting*.)  As a simple illustration,  consider the case where your set of example objects was `{ 'ant', 'elephant'}`.  In this case, the question "is it smaller than a typical adult human" would enable you to differentiate between examples optimally.   However, that question would be useless if our set of example objects was the set of all domesticated dog breeds.

[Random decision forest models](https://en.wikipedia.org/wiki/Random_forest) work by training an *ensemble* of imprecise decision trees that only consider subsets of features or examples and then aggregating the results from the ensemble.  By learning and aggregating an ensemble of trees, random decision forests can be more accurate than individual decision trees *and* are less likely to overfit.  In this notebook, we'll use a random decision forest to classify documents as either "spam" (based on food reviews) or "legitimate" (based on Jane Austen).

We'll begin by loading in the feature vectors which we generated in either [the simple summaries feature extraction notebook](03-feature-engineering-summaries.ipynb) or [the TF-IDF feature extraction notebook](03-feature-engineering-tfidf.ipynb). 

In [1]:
import pandas as pd
import os.path

features = pd.read_parquet(os.path.join("data", "features_tfidf.parquet"))

When doing exploratory analysis, it's often a good idea to inspect your data as a sanity check.  In this case, we'll make sure that the feature vectors we generated in the last notebook have the shape we expect!

In [2]:
features.sample(5)

,index,label,0,1,2,3,4,5,6,7,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
29356,9356,spam,0.0,0.0,0.0,0.186592,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10979,10979,legitimate,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19034,19034,legitimate,0.0,0.0,0.0,0.000000,0.085985,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21554,1554,spam,0.0,0.0,0.0,0.000000,0.000000,0.0,0.148087,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34727,14727,spam,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection

train, test = model_selection.train_test_split(features)

rfc = RandomForestClassifier(n_estimators=25, random_state=404)

In [4]:
MAX_FEATURES=512
rfc.fit(X=train.iloc[:,2:train.shape[1]], y=train["label"])

RandomForestClassifier(n_estimators=25, random_state=404)

In [5]:
from mlworkflows import plot

predictions = rfc.predict(test.iloc[:,2:train.shape[1]])
df, chart = plot.binary_confusion_matrix(test["label"], predictions)

In [6]:
chart

alt.Chart(...)

We can look at the raw numbers, and proportions of correctly and incorrectly classified items: 

In [7]:
df

,predicted,actual,raw_count,value
0,legitimate,legitimate,4604,0.912044
1,legitimate,spam,444,0.087956
2,spam,legitimate,240,0.048465
3,spam,spam,4712,0.951535


We can also look at the precision, recall and f1-score for the model. 

In [8]:
from sklearn.metrics import classification_report
print(classification_report(test.label.values, predictions))

              precision    recall  f1-score   support

  legitimate       0.95      0.91      0.93      5048
        spam       0.91      0.95      0.93      4952

    accuracy                           0.93     10000
   macro avg       0.93      0.93      0.93     10000
weighted avg       0.93      0.93      0.93     10000



One interesting aspect of random decision forests is that they provide a metric for how important each feature was to the ultimate conclusion.  This is a useful property both for having *explainable models* (i.e., so you can explain to a human why the model made a particular prediction) and for guiding further experiments (i.e., so you can learn more about the real world based on what the model has identified as likely to be correlated with what you're trying to predict).

In [9]:
l = list(enumerate(rfc.feature_importances_))

In [10]:
l.sort(key=lambda x: -x[1])
l[:20]

[(228, 0.08642785085541246),
 (259, 0.030584767235343605),
 (749, 0.03028573281827092),
 (797, 0.02946455608787317),
 (78, 0.026820481554375872),
 (182, 0.024693875896320296),
 (913, 0.024452908139693332),
 (512, 0.02116792860938632),
 (260, 0.020684938234151726),
 (365, 0.01857868836747351),
 (360, 0.016517979141711048),
 (894, 0.013720872466744881),
 (895, 0.012969168184069515),
 (62, 0.012647852182448049),
 (384, 0.012419086750264381),
 (300, 0.011752260967708698),
 (162, 0.010762032706844813),
 (710, 0.010437744738941055),
 (532, 0.00983374695187193),
 (1003, 0.009281002306516982)]

What these features actually *mean* depends on which feature engineering approach you chose in the last notebook.  In the case of the simple summaries approach, it's fairly straightforward -- we can just take the column names:

In [11]:
if len(l) < 20:
    # the simple summaries have fewer than 20 features
    d = dict(enumerate(train.columns[2:]))
    for k, v in l[:20]:
        print(d[k], v)

If you used the simple summaries approach, you'll probably see that `stop_words` is the most important feature.

If you used the tf-idf approach, the preceding cell shouldn't have produced any output and we'll need a more involved approach.  Recall that we used [feature hashing](https://en.wikipedia.org/wiki/Feature_hashing) for the tf-idf feature vectors.  Feature hashing has many benefits, but one of the downsides is that hashing is one-way -- we can't go from the entry in the feature vector to the word that produced it.  However, we can do a little extra work to try and reconstruct the words that _might_ have produced such a vector.

The size of our feature vectors is smaller than the number of words we might have seen in our corpus of documents, [which means that more than one word will be represented by counts in a given bucket](https://en.wikipedia.org/wiki/Pigeonhole_principle).  But we can construct a list of the words that may have mapped to each bucket by getting a vocabulary for our document corpus and then hashing each word, which we can use to identify which words _may_ have corresponded to the most important features.

In [12]:
if len(l) > 20:
    from sklearn.utils.murmurhash import murmurhash3_bytes_s32
    from sklearn.feature_extraction.text import CountVectorizer
    from collections import defaultdict
    
    def fhash(v, size = 1024):
        return murmurhash3_bytes_s32(v.encode("utf-8"), 0) % size
    data = pd.read_parquet("data/training.parquet")
    vectorizer = CountVectorizer(token_pattern='(?u)\\b[A-Za-z]\\w+\\b')
    vectorizer.fit(data["text"])
    d = defaultdict(lambda: [])
    for k in vectorizer.vocabulary_.keys():
        d[fhash(k)].append(k)
    
    for k, v in l[:20]:
        print(d[k], v)

['honour', 'most', 'sunflower', 'silence', 'deprived', 'explore', 'fourteen', 'burger', 'environment', 'tested', 'uske', 'un', 'paced', 'madeira', 'gental', 'fulls', 'harmonised', 'apeice', 'bets', 'admonitions', 'mantienen', 'dragonwell', 'fresca', 'heimlich', 'han', 'accusing', 'untested', 'countertop', 'durham', 'sunrise', 'rushmore', 'willow', 'permeatedthe', 'minsthe', 'machince'] 0.08642785085541246
['he', 'imaginary', 'advisable', 'complain', 'separation', 'meeting', 'blue', 'wretched', 'faded', 'morrows', 'seduced', 'rested', 'deceptive', 'flawness', 'concessions', 'combat', 'iniquity', 'shea', 'tbs', 'relenting', 'retrievers', 'horde', 'whiskey', 'weim', 'caters', 'waruva', 'gullible', 'optimal', 'protability', 'smash', 'db', 'nearlyas', 'endorsed', 'tailgate', 'artic', 'onem', 'bargin', 'colombians', 'htm', 'fabricate'] 0.030584767235343605
['she', 'calcium', 'shut', 'constancy', 'explanation', 'wink', 'wandered', 'clair', 'shots', 'fiftieth', 'effective', 'archbishop', 'dest

This sort of investigation is useful to see how well our feature count is working in the context of feature hashing:  if we see that an important feature has many apparently-unrelated words (especially if many relatively-common words are in the same bucket), we may want to increase the number of buckets to improve the performance of our model.

Finally, we'll want to save the model so that we can use it outside of this notebook.

In [13]:
from mlworkflows import util
util.serialize_to(rfc, "model_tfidf_forest.sav")